# Process the EIC Load Data for the NTP Heat Wave Grid Stress Events

In [ ]:
# Start by importing the packages we need:
import os
import glob
import datetime
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Set the Directory Structure

In [ ]:
# Identify the data input and image output directory:
data_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave_loads/data/'
plot_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave_loads/plots/'


## Suppress Future Warnings


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


## Create a Function to Process the 2035 GridView Data Used in Scaling


In [ ]:
def process_gridview_data(data_dir: str):
    
    # Read in the raw data .csv file:
    gv_df = pd.read_csv((data_dir + 'wecc_and_eic_load_2035.csv'), low_memory=False)
    
    # Subset to just the annual total demand by BA:
    gv_df = gv_df[-3:-2]
    
    # Strip the unecessary bits from the column names:
    gv_df.columns = gv_df.columns.str.replace("_2035.dat", "")
    gv_df.columns = gv_df.columns.str.replace("Load_", "")
       
    # Delete the index and last column:
    del gv_df["Index"]
    
    # Convert the dataframe to float:
    gv_df = gv_df.astype(float)
    
    # Compute the total loads for the BAs with subregions:
    gv_df['CISO'] = (gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD'] + gv_df['VEA']).round(2)
    gv_df['FMPP_Sum'] = (gv_df['FTP'] + gv_df['KEY'] + gv_df['LWU'] + gv_df['FMPP'] + gv_df['RCU'] + gv_df['TCEC']).round(2)
    gv_df['FPL_Sum'] = (gv_df['FPL'] + gv_df['OLEANDER']).round(2)
    gv_df['IPCO'] = (gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV']).round(2)
    gv_df['MISO'] = (gv_df['EES-EMI'] + gv_df['EES-EAI'] + gv_df['HE'] + gv_df['DEI'] + gv_df['SIGE'] + gv_df['IPL'] + gv_df['NIPS'] + gv_df['METC'] + 
                     gv_df['ITCT'] + gv_df['WEC'] + gv_df['MIUP'] + gv_df['BREC'] + gv_df['HMPL'] + gv_df['LAGN'] + 
                     gv_df['CWLD'] + gv_df['SMEPA'] + gv_df['EES'] + gv_df['AMMO'] + gv_df['AMIL'] + gv_df['CWLP'] + 
                     gv_df['SIPC'] + gv_df['OMUA'] + gv_df['CLEC'] + gv_df['LAFA'] + gv_df['LEPA'] + gv_df['MJMEUC'] + 
                     gv_df['SWPA'] + gv_df['XEL'] + gv_df['MP'] + gv_df['SMMPA'] + gv_df['GRE'] + gv_df['OTP'] + gv_df['ALTW'] + 
                     gv_df['MEC'] + gv_df['HAST'] + gv_df['LES'] + gv_df['MDU'] + gv_df['BEPC-MISO'] + gv_df['DPC'] + gv_df['ALTE'] + 
                     gv_df['WPS'] + gv_df['MGE'] + gv_df['UPPC']).round(2)
    gv_df['NEVP_Sum'] = (gv_df['NEVP'] + gv_df['SPPC']).round(2)
    gv_df['PACE'] = (gv_df['PAID'] + gv_df['PAUT'] + gv_df['PAWY']).round(2)
    gv_df['PJM_Sum'] = (gv_df['APS'] + gv_df['ATSI'] + gv_df['AEP'] + gv_df['OVEC'] + gv_df['DAY'] + gv_df['DEO&K'] + gv_df['DLCO'] + 
                        gv_df['CE'] + gv_df['PJM'] + gv_df['PENELEC'] + gv_df['ME'] + gv_df['JCPL'] + gv_df['PL'] + gv_df['PECO'] + 
                        gv_df['PSEG'] + gv_df['BGE'] + gv_df['PEPCO'] + gv_df['AE'] + gv_df['DP&L'] + gv_df['UGI'] + gv_df['RECO'] + 
                        gv_df['SMECO'] + gv_df['EKPC'] + gv_df['DVP']).round(2)
    gv_df['SOCO_Sum'] = (gv_df['SOCO'] + gv_df['GULF']).round(2)
    
    gv_df['SWPP'] = (gv_df['AEPW'] + gv_df['GRDA'] + gv_df['OKGE'] + gv_df['WFEC'] + gv_df['SPS'] + 
                     gv_df['OMPA'] + gv_df['MIDW'] + gv_df['SUNC'] + gv_df['WERE'] + gv_df['KCPL'] + gv_df['KACY'] + gv_df['EMDE'] + 
                     gv_df['INDN'] + gv_df['SPRM'] + gv_df['NPPD'] + gv_df['GRIS'] + gv_df['OPPD'] + gv_df['WAPA'] + gv_df['BEPC-SPP']).round(2)
    gv_df['TEC'] = (gv_df['TECO']).round(2)
    gv_df['TVA_Sum'] = (gv_df['TVA'] + gv_df['SMT']).round(2)
    
    # Rename a few columns for consistency:
    gv_df.rename(columns={'FMPP': 'FMPP_FMPP', 'FPL': 'FPL_FPL', 'NEVP': 'NEVP_NEVP', 'PJM': 'PJM_PJM', 'SOCO': 'SOCO_SOCO', 'TVA': 'TVA_TVA'}, inplace=True)                 
    gv_df.rename(columns={'FMPP_Sum': 'FMPP', 'FPL_Sum': 'FPL', 'NEVP_Sum': 'NEVP', 'PJM_Sum': 'PJM', 'SOCO_Sum': 'SOCO', 'TVA_Sum': 'TVA'}, inplace=True) 
    
    # Rename a few columns so that they match the TELL names:
    gv_df.rename(columns={'DEF': 'FPC', 'ISO-NE': 'ISNE', 'NYISO': 'NYIS', 'ERCOT': 'ERCO'}, inplace=True) 
    
    # Squeeze the dataframe:
    gv_df = gv_df.squeeze().to_frame()
        
    # Rename the columns:
    gv_df.reset_index(inplace=True)
    gv_df = gv_df.rename(columns = {'index':'BA'})
    gv_df.rename(columns={gv_df.columns[1]: "GV_Total_Load_MWh" }, inplace = True)
       
    # Sort the dataframe alphabetically by BA name:
    gv_df = gv_df.sort_values('BA')
       
    # Return the output dataframe:
    return gv_df


In [ ]:
gv_df = process_gridview_data(data_dir = data_dir)

gv_df


## Create a Function to Aggregate the Raw TELL MLP Output into a Single Dataframe:


In [ ]:
def aggregate_mlp_output_files(data_dir: str, year_to_process: str):
    
    # Create a list of all of the MLP output files in the "mlp_input_dir" and aggregate the files in that list:
    list_of_files = sorted(glob.glob(os.path.join(data_dir, 'TELL_Data', year_to_process, '*_mlp_output.csv')))

    # Loop over the list of MLP output files:
    for file in range(len(list_of_files)):

        # Read in the .csv file and replace missing values with nan:
        mlp_data = pd.read_csv(list_of_files[file]).replace(-9999, np.nan)

        # Rename the "Load" variable:
        mlp_data.rename(columns={'Load': 'Hourly_Load_MWh'}, inplace=True)

        # Replacing missing or negative loads with NaN:
        mlp_data.loc[~(mlp_data['Hourly_Load_MWh'] > 0), 'Hourly_Load_MWh'] = np.nan

        # Aggregate the output into a new dataframe:
        if file == 0:
            tell_df = mlp_data
        else:
            tell_df = pd.concat([tell_df, mlp_data])
    
    # Return the output dataframe:
    return tell_df


## Create a Function to Scale the TELL Output Based on the GridView 2035 Values:


In [ ]:
def scale_tell_loads(data_dir: str, year_to_process: str):
    
    # Aggregate the TELL MLP files:
    tell_df = aggregate_mlp_output_files(data_dir = data_dir,
                                         year_to_process = year_to_process)
    
    # Read in the processed GridView file and rename a column for consistency:
    gv_df = process_gridview_data(data_dir = data_dir)
        
    # Merge the tell_df and gv_df dataframes based on common BA names:
    merged_df = tell_df.merge(gv_df, on=['BA'])
    
    # Sum the hourly TELL loads by BA into annual total loads:
    merged_df['TELL_Total_Load_MWh'] = merged_df.groupby('BA')['Hourly_Load_MWh'].transform('sum')
    
    # Compute the scaling factors that force the annual total loads to agree:
    merged_df['Scaling_Factor'] = merged_df['GV_Total_Load_MWh'] / merged_df['TELL_Total_Load_MWh']
    
    # Compute the scaled hourly loads:
    merged_df['Hourly_Load_MWh_Scaled'] = merged_df['Hourly_Load_MWh'] * merged_df['Scaling_Factor']
    
    # Compute the hours since the start of the year:
    merged_df['Hour'] = ((pd.to_datetime(merged_df['Time_UTC']) - datetime.datetime(int(year_to_process), 1, 1, 0, 0, 0)) / np.timedelta64(1, 'h') + 1).astype(int)
    
    # Only keep the columns that are needed:
    scaled_tell_df = merged_df[['Hour', 'BA', 'Hourly_Load_MWh_Scaled']].copy()
    
    # Drop the rows with missing values (i.e., there is not a corresponding GridView load):
    scaled_tell_df = scaled_tell_df.dropna(how = 'any')
    
    # Rename the load variable and round it to 5 decimals:
    scaled_tell_df.rename(columns={'Hourly_Load_MWh_Scaled': 'Load_MWh'}, inplace=True)
    scaled_tell_df['Load_MWh'] = scaled_tell_df['Load_MWh'].round(5)
    
    # Return the output dataframe:
    return scaled_tell_df


In [ ]:
# Aggregate and scale the TELL MLP files:
scaled_tell_df = scale_tell_loads(data_dir = data_dir, 
                                  year_to_process = '2055')

scaled_tell_df


## Create a Function to Format the Output for Ingest to GridView:


In [ ]:
def format_scaled_tell_loads(data_dir: str, year_to_process: str):
    
    # Process the GridView file:
    gv_df = process_gridview_data(data_dir = data_dir)
    
    # Compute the load fractions for the CISO subregions:
    CIPB_LF = (gv_df.loc[(gv_df['BA'] == 'CIPB')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['GV_Total_Load_MWh'].values[0])
    CIPV_LF = (gv_df.loc[(gv_df['BA'] == 'CIPV')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['GV_Total_Load_MWh'].values[0])
    CISC_LF = (gv_df.loc[(gv_df['BA'] == 'CISC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['GV_Total_Load_MWh'].values[0])
    CISD_LF = (gv_df.loc[(gv_df['BA'] == 'CISD')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['GV_Total_Load_MWh'].values[0])
    VEA_LF  = (gv_df.loc[(gv_df['BA'] == 'VEA' )]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the IPCO subregions:
    IPFE_LF = (gv_df.loc[(gv_df['BA'] == 'IPFE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['GV_Total_Load_MWh'].values[0])
    IPMV_LF = (gv_df.loc[(gv_df['BA'] == 'IPMV')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['GV_Total_Load_MWh'].values[0])
    IPTV_LF = (gv_df.loc[(gv_df['BA'] == 'IPTV')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the NEVP subregions:
    NEVP_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_NEVP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['GV_Total_Load_MWh'].values[0])
    SPPC_LF = (gv_df.loc[(gv_df['BA'] == 'SPPC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the PACE subregions:
    PAID_LF = (gv_df.loc[(gv_df['BA'] == 'PAID')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['GV_Total_Load_MWh'].values[0])
    PAUT_LF = (gv_df.loc[(gv_df['BA'] == 'PAUT')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['GV_Total_Load_MWh'].values[0])
    PAWY_LF = (gv_df.loc[(gv_df['BA'] == 'PAWY')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the FMPP subregions:
    FTP_LF = (gv_df.loc[(gv_df['BA'] == 'FTP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FMPP')]['GV_Total_Load_MWh'].values[0])
    KEY_LF = (gv_df.loc[(gv_df['BA'] == 'KEY')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FMPP')]['GV_Total_Load_MWh'].values[0])
    LWU_LF = (gv_df.loc[(gv_df['BA'] == 'LWU')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FMPP')]['GV_Total_Load_MWh'].values[0])
    FMPP_LF = (gv_df.loc[(gv_df['BA'] == 'FMPP_FMPP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FMPP')]['GV_Total_Load_MWh'].values[0])
    RCU_LF = (gv_df.loc[(gv_df['BA'] == 'RCU')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FMPP')]['GV_Total_Load_MWh'].values[0])
    TCEC_LF = (gv_df.loc[(gv_df['BA'] == 'TCEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FMPP')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the FPL subregions:
    FPL_LF = (gv_df.loc[(gv_df['BA'] == 'FPL_FPL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FPL')]['GV_Total_Load_MWh'].values[0])
    OLEANDER_LF = (gv_df.loc[(gv_df['BA'] == 'OLEANDER')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'FPL')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the MISO subregions:
    EES_EMI_LF = (gv_df.loc[(gv_df['BA'] == 'EES-EMI')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    EES_EAI_LF = (gv_df.loc[(gv_df['BA'] == 'EES-EAI')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    HE_LF = (gv_df.loc[(gv_df['BA'] == 'HE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    DEI_LF = (gv_df.loc[(gv_df['BA'] == 'DEI')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    SIGE_LF = (gv_df.loc[(gv_df['BA'] == 'SIGE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    IPL_LF = (gv_df.loc[(gv_df['BA'] == 'IPL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    NIPS_LF = (gv_df.loc[(gv_df['BA'] == 'NIPS')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    METC_LF = (gv_df.loc[(gv_df['BA'] == 'METC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    ITCT_LF = (gv_df.loc[(gv_df['BA'] == 'ITCT')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    WEC_LF = (gv_df.loc[(gv_df['BA'] == 'WEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    MIUP_LF = (gv_df.loc[(gv_df['BA'] == 'MIUP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    BREC_LF = (gv_df.loc[(gv_df['BA'] == 'BREC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    HMPL_LF = (gv_df.loc[(gv_df['BA'] == 'HMPL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    LAGN_LF = (gv_df.loc[(gv_df['BA'] == 'LAGN')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    CWLD_LF = (gv_df.loc[(gv_df['BA'] == 'CWLD')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    SMEPA_LF = (gv_df.loc[(gv_df['BA'] == 'SMEPA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    EES_LF = (gv_df.loc[(gv_df['BA'] == 'EES')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    AMMO_LF = (gv_df.loc[(gv_df['BA'] == 'AMMO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    AMIL_LF = (gv_df.loc[(gv_df['BA'] == 'AMIL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    CWLP_LF = (gv_df.loc[(gv_df['BA'] == 'CWLP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    SIPC_LF = (gv_df.loc[(gv_df['BA'] == 'SIPC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    OMUA_LF = (gv_df.loc[(gv_df['BA'] == 'OMUA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    CLEC_LF = (gv_df.loc[(gv_df['BA'] == 'CLEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    LAFA_LF = (gv_df.loc[(gv_df['BA'] == 'LAFA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    LEPA_LF = (gv_df.loc[(gv_df['BA'] == 'LEPA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    MJMEUC_LF = (gv_df.loc[(gv_df['BA'] == 'MJMEUC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    SWPA_LF = (gv_df.loc[(gv_df['BA'] == 'SWPA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    XEL_LF = (gv_df.loc[(gv_df['BA'] == 'XEL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    MP_LF = (gv_df.loc[(gv_df['BA'] == 'MP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    SMMPA_LF = (gv_df.loc[(gv_df['BA'] == 'SMMPA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    GRE_LF = (gv_df.loc[(gv_df['BA'] == 'GRE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    OTP_LF = (gv_df.loc[(gv_df['BA'] == 'OTP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    ALTW_LF = (gv_df.loc[(gv_df['BA'] == 'ALTW')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    MEC_LF = (gv_df.loc[(gv_df['BA'] == 'MEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    HAST_LF = (gv_df.loc[(gv_df['BA'] == 'HAST')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    LES_LF = (gv_df.loc[(gv_df['BA'] == 'LES')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    MDU_LF = (gv_df.loc[(gv_df['BA'] == 'MDU')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    BEPC_MISO_LF = (gv_df.loc[(gv_df['BA'] == 'BEPC-MISO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    DPC_LF = (gv_df.loc[(gv_df['BA'] == 'DPC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    ALTE_LF = (gv_df.loc[(gv_df['BA'] == 'ALTE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    WPS_LF = (gv_df.loc[(gv_df['BA'] == 'WPS')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    MGE_LF = (gv_df.loc[(gv_df['BA'] == 'MGE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    UPPC_LF = (gv_df.loc[(gv_df['BA'] == 'UPPC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'MISO')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the PJM subregions:
    APS_LF = (gv_df.loc[(gv_df['BA'] == 'APS')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    ATSI_LF = (gv_df.loc[(gv_df['BA'] == 'ATSI')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    AEP_LF = (gv_df.loc[(gv_df['BA'] == 'AEP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    OVEC_LF = (gv_df.loc[(gv_df['BA'] == 'OVEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    DAY_LF = (gv_df.loc[(gv_df['BA'] == 'DAY')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    DEOK_LF = (gv_df.loc[(gv_df['BA'] == 'DEO&K')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    DLCO_LF = (gv_df.loc[(gv_df['BA'] == 'DLCO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    CE_LF = (gv_df.loc[(gv_df['BA'] == 'CE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    PJM_LF = (gv_df.loc[(gv_df['BA'] == 'PJM_PJM')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    PENELEC_LF = (gv_df.loc[(gv_df['BA'] == 'PENELEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    ME_LF = (gv_df.loc[(gv_df['BA'] == 'ME')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    JCPL_LF = (gv_df.loc[(gv_df['BA'] == 'JCPL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    PL_LF = (gv_df.loc[(gv_df['BA'] == 'PL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    PECO_LF = (gv_df.loc[(gv_df['BA'] == 'PECO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    PSEG_LF = (gv_df.loc[(gv_df['BA'] == 'PSEG')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    BGE_LF = (gv_df.loc[(gv_df['BA'] == 'BGE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    PEPCO_LF = (gv_df.loc[(gv_df['BA'] == 'PEPCO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    AE_LF = (gv_df.loc[(gv_df['BA'] == 'AE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    DPL_LF = (gv_df.loc[(gv_df['BA'] == 'DP&L')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    UGI_LF = (gv_df.loc[(gv_df['BA'] == 'UGI')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    RECO_LF = (gv_df.loc[(gv_df['BA'] == 'RECO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    SMECO_LF = (gv_df.loc[(gv_df['BA'] == 'SMECO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    EKPC_LF = (gv_df.loc[(gv_df['BA'] == 'EKPC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    DVP_LF = (gv_df.loc[(gv_df['BA'] == 'DVP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PJM')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the SOCO subregions:
    SOCO_LF = (gv_df.loc[(gv_df['BA'] == 'SOCO_SOCO')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SOCO')]['GV_Total_Load_MWh'].values[0])
    GULF_LF = (gv_df.loc[(gv_df['BA'] == 'GULF')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SOCO')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the SWPP subregions:
    AEPW_LF = (gv_df.loc[(gv_df['BA'] == 'AEPW')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    GRDA_LF = (gv_df.loc[(gv_df['BA'] == 'GRDA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    OKGE_LF = (gv_df.loc[(gv_df['BA'] == 'OKGE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    WFEC_LF = (gv_df.loc[(gv_df['BA'] == 'WFEC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    SPS_LF = (gv_df.loc[(gv_df['BA'] == 'SPS')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    OMPA_LF = (gv_df.loc[(gv_df['BA'] == 'OMPA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    MIDW_LF = (gv_df.loc[(gv_df['BA'] == 'MIDW')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    SUNC_LF = (gv_df.loc[(gv_df['BA'] == 'SUNC')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    WERE_LF = (gv_df.loc[(gv_df['BA'] == 'WERE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    KCPL_LF = (gv_df.loc[(gv_df['BA'] == 'KCPL')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    KACY_LF = (gv_df.loc[(gv_df['BA'] == 'KACY')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    EMDE_LF = (gv_df.loc[(gv_df['BA'] == 'EMDE')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    INDN_LF = (gv_df.loc[(gv_df['BA'] == 'INDN')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    SPRM_LF = (gv_df.loc[(gv_df['BA'] == 'SPRM')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    NPPD_LF = (gv_df.loc[(gv_df['BA'] == 'NPPD')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    GRIS_LF = (gv_df.loc[(gv_df['BA'] == 'GRIS')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    OPPD_LF = (gv_df.loc[(gv_df['BA'] == 'OPPD')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    WAPA_LF = (gv_df.loc[(gv_df['BA'] == 'WAPA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    BEPC_SPP_LF = (gv_df.loc[(gv_df['BA'] == 'BEPC-SPP')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'SWPP')]['GV_Total_Load_MWh'].values[0])
    
    # Compute the load fractions for the TVA subregions:
    TVA_LF = (gv_df.loc[(gv_df['BA'] == 'TVA_TVA')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'TVA')]['GV_Total_Load_MWh'].values[0])
    SMT_LF = (gv_df.loc[(gv_df['BA'] == 'SMT')]['GV_Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'TVA')]['GV_Total_Load_MWh'].values[0])
    
    # Aggregate the TELL MLP files:
    scaled_tell_df = scale_tell_loads(data_dir = data_dir, 
                                      year_to_process = year_to_process)
   
    # Reshape the dataframe and drop the indexes:
    load_df = scaled_tell_df.pivot(index = 'Hour', columns = 'BA', values = 'Load_MWh')
    load_df = load_df.reset_index(drop=False)
    
    # Add back in the text to the column headers:
    load_df = load_df.add_suffix('_2035.dat')
    load_df = load_df.add_prefix('Load_')
    
    # Rename the time variable:
    load_df.rename(columns={'Load_Hour_2035.dat': 'Index'}, inplace=True)
    
    # Compute the loads for CISO subregions:
    load_df['Load_CIPB_2035.dat'] = load_df['Load_CISO_2035.dat'] * CIPB_LF
    load_df['Load_CIPV_2035.dat'] = load_df['Load_CISO_2035.dat'] * CIPV_LF
    load_df['Load_CISC_2035.dat'] = load_df['Load_CISO_2035.dat'] * CISC_LF
    load_df['Load_CISD_2035.dat'] = load_df['Load_CISO_2035.dat'] * CISD_LF
    load_df['Load_VEA_2035.dat'] = load_df['Load_CISO_2035.dat'] * VEA_LF
    
    # Compute the loads for IPCO subregions:
    load_df['Load_IPFE_2035.dat'] = load_df['Load_IPCO_2035.dat'] * IPFE_LF
    load_df['Load_IPMV_2035.dat'] = load_df['Load_IPCO_2035.dat'] * IPMV_LF
    load_df['Load_IPTV_2035.dat'] = load_df['Load_IPCO_2035.dat'] * IPTV_LF
    
    # Compute the loads for NEVP subregions:
    load_df['Load_NEVP_Temp_2035.dat'] = load_df['Load_NEVP_2035.dat'] * NEVP_LF
    load_df['Load_SPPC_2035.dat'] = load_df['Load_NEVP_2035.dat'] * SPPC_LF
    
    # Compute the loads for PACE subregions:
    load_df['Load_PAID_2035.dat'] = load_df['Load_PACE_2035.dat'] * PAID_LF
    load_df['Load_PAUT_2035.dat'] = load_df['Load_PACE_2035.dat'] * PAUT_LF
    load_df['Load_PAWY_2035.dat'] = load_df['Load_PACE_2035.dat'] * PAWY_LF
    
    # Compute the loads for the FMPP subregions:
    load_df['Load_FTP_2035.dat'] = load_df['Load_FMPP_2035.dat'] * FTP_LF
    load_df['Load_KEY_2035.dat'] = load_df['Load_FMPP_2035.dat'] * KEY_LF
    load_df['Load_LWU_2035.dat'] = load_df['Load_FMPP_2035.dat'] * LWU_LF
    load_df['Load_FMPP_Temp_2035.dat'] = load_df['Load_FMPP_2035.dat'] * FMPP_LF
    load_df['Load_RCU_2035.dat'] = load_df['Load_FMPP_2035.dat'] * RCU_LF
    load_df['Load_TCEC_2035.dat'] = load_df['Load_FMPP_2035.dat'] * TCEC_LF
    
    # Compute the loads for the FPL subregions:
    load_df['Load_FPL_Temp_2035.dat'] = load_df['Load_FPL_2035.dat'] * FPL_LF
    load_df['Load_OLEANDER_2035.dat'] = load_df['Load_FPL_2035.dat'] * OLEANDER_LF
    
    # Compute the loads for the MISO subregions:
    load_df['Load_EES-EMI_2035.dat'] = load_df['Load_MISO_2035.dat'] * EES_EMI_LF
    load_df['Load_EES-EAI_2035.dat'] = load_df['Load_MISO_2035.dat'] * EES_EAI_LF
    load_df['Load_HE_2035.dat'] = load_df['Load_MISO_2035.dat'] * HE_LF
    load_df['Load_DEI_2035.dat'] = load_df['Load_MISO_2035.dat'] * DEI_LF
    load_df['Load_SIGE_2035.dat'] = load_df['Load_MISO_2035.dat'] * SIGE_LF
    load_df['Load_IPL_2035.dat'] = load_df['Load_MISO_2035.dat'] * IPL_LF
    load_df['Load_NIPS_2035.dat'] = load_df['Load_MISO_2035.dat'] * NIPS_LF
    load_df['Load_METC_2035.dat'] = load_df['Load_MISO_2035.dat'] * METC_LF
    load_df['Load_ITCT_2035.dat'] = load_df['Load_MISO_2035.dat'] * ITCT_LF
    load_df['Load_WEC_2035.dat'] = load_df['Load_MISO_2035.dat'] * WEC_LF
    load_df['Load_MIUP_2035.dat'] = load_df['Load_MISO_2035.dat'] * MIUP_LF
    load_df['Load_BREC_2035.dat'] = load_df['Load_MISO_2035.dat'] * BREC_LF
    load_df['Load_HMPL_2035.dat'] = load_df['Load_MISO_2035.dat'] * HMPL_LF
    load_df['Load_LAGN_2035.dat'] = load_df['Load_MISO_2035.dat'] * LAGN_LF
    load_df['Load_CWLD_2035.dat'] = load_df['Load_MISO_2035.dat'] * CWLD_LF
    load_df['Load_SMEPA_2035.dat'] = load_df['Load_MISO_2035.dat'] * SMEPA_LF
    load_df['Load_EES_2035.dat'] = load_df['Load_MISO_2035.dat'] * EES_LF
    load_df['Load_AMMO_2035.dat'] = load_df['Load_MISO_2035.dat'] * AMMO_LF
    load_df['Load_AMIL_2035.dat'] = load_df['Load_MISO_2035.dat'] * AMIL_LF
    load_df['Load_CWLP_2035.dat'] = load_df['Load_MISO_2035.dat'] * CWLP_LF
    load_df['Load_SIPC_2035.dat'] = load_df['Load_MISO_2035.dat'] * SIPC_LF
    load_df['Load_OMUA_2035.dat'] = load_df['Load_MISO_2035.dat'] * OMUA_LF
    load_df['Load_CLEC_2035.dat'] = load_df['Load_MISO_2035.dat'] * CLEC_LF
    load_df['Load_LAFA_2035.dat'] = load_df['Load_MISO_2035.dat'] * LAFA_LF
    load_df['Load_LEPA_2035.dat'] = load_df['Load_MISO_2035.dat'] * LEPA_LF
    load_df['Load_MJMEUC_2035.dat'] = load_df['Load_MISO_2035.dat'] * MJMEUC_LF
    load_df['Load_SWPA_2035.dat'] = load_df['Load_MISO_2035.dat'] * SWPA_LF
    load_df['Load_XEL_2035.dat'] = load_df['Load_MISO_2035.dat'] * XEL_LF
    load_df['Load_MP_2035.dat'] = load_df['Load_MISO_2035.dat'] * MP_LF
    load_df['Load_SMMPA_2035.dat'] = load_df['Load_MISO_2035.dat'] * SMMPA_LF
    load_df['Load_GRE_2035.dat'] = load_df['Load_MISO_2035.dat'] * GRE_LF
    load_df['Load_OTP_2035.dat'] = load_df['Load_MISO_2035.dat'] * OTP_LF
    load_df['Load_ALTW_2035.dat'] = load_df['Load_MISO_2035.dat'] * ALTW_LF
    load_df['Load_MEC_2035.dat'] = load_df['Load_MISO_2035.dat'] * MEC_LF
    load_df['Load_HAST_2035.dat'] = load_df['Load_MISO_2035.dat'] * HAST_LF
    load_df['Load_LES_2035.dat'] = load_df['Load_MISO_2035.dat'] * LES_LF
    load_df['Load_MDU_2035.dat'] = load_df['Load_MISO_2035.dat'] * MDU_LF
    load_df['Load_BEPC-MISO_2035.dat'] = load_df['Load_MISO_2035.dat'] * BEPC_MISO_LF
    load_df['Load_DPC_2035.dat'] = load_df['Load_MISO_2035.dat'] * DPC_LF
    load_df['Load_ALTE_2035.dat'] = load_df['Load_MISO_2035.dat'] * ALTE_LF
    load_df['Load_WPS_2035.dat'] = load_df['Load_MISO_2035.dat'] * WPS_LF
    load_df['Load_MGE_2035.dat'] = load_df['Load_MISO_2035.dat'] * MGE_LF
    load_df['Load_UPPC_2035.dat'] = load_df['Load_MISO_2035.dat'] * UPPC_LF
    
    # Compute the loads for the PJM subregions:
    load_df['Load_APS_2035.dat'] = load_df['Load_PJM_2035.dat'] * APS_LF
    load_df['Load_ATSI_2035.dat'] = load_df['Load_PJM_2035.dat'] * ATSI_LF
    load_df['Load_AEP_2035.dat'] = load_df['Load_PJM_2035.dat'] * AEP_LF
    load_df['Load_OVEC_2035.dat'] = load_df['Load_PJM_2035.dat'] * OVEC_LF
    load_df['Load_DAY_2035.dat'] = load_df['Load_PJM_2035.dat'] * DAY_LF
    load_df['Load_DEO&K_2035.dat'] = load_df['Load_PJM_2035.dat'] * DEOK_LF
    load_df['Load_DLCO_2035.dat'] = load_df['Load_PJM_2035.dat'] * DLCO_LF
    load_df['Load_CE_2035.dat'] = load_df['Load_PJM_2035.dat'] * CE_LF
    load_df['Load_PJM_Temp_2035.dat'] = load_df['Load_PJM_2035.dat'] * PJM_LF
    load_df['Load_PENELEC_2035.dat'] = load_df['Load_PJM_2035.dat'] * PENELEC_LF
    load_df['Load_ME_2035.dat'] = load_df['Load_PJM_2035.dat'] * ME_LF
    load_df['Load_JCPL_2035.dat'] = load_df['Load_PJM_2035.dat'] * JCPL_LF
    load_df['Load_PL_2035.dat'] = load_df['Load_PJM_2035.dat'] * PL_LF
    load_df['Load_PECO_2035.dat'] = load_df['Load_PJM_2035.dat'] * PECO_LF
    load_df['Load_PSEG_2035.dat'] = load_df['Load_PJM_2035.dat'] * PSEG_LF
    load_df['Load_BGE_2035.dat'] = load_df['Load_PJM_2035.dat'] * BGE_LF
    load_df['Load_PEPCO_2035.dat'] = load_df['Load_PJM_2035.dat'] * PEPCO_LF
    load_df['Load_AE_2035.dat'] = load_df['Load_PJM_2035.dat'] * AE_LF
    load_df['Load_DP&L_2035.dat'] = load_df['Load_PJM_2035.dat'] * DPL_LF
    load_df['Load_UGI_2035.dat'] = load_df['Load_PJM_2035.dat'] * UGI_LF
    load_df['Load_RECO_2035.dat'] = load_df['Load_PJM_2035.dat'] * RECO_LF
    load_df['Load_SMECO_2035.dat'] = load_df['Load_PJM_2035.dat'] * SMECO_LF
    load_df['Load_EKPC_2035.dat'] = load_df['Load_PJM_2035.dat'] * EKPC_LF
    load_df['Load_DVP_2035.dat'] = load_df['Load_PJM_2035.dat'] * DVP_LF
    
    # Compute the loads for the SOCO subregions:
    load_df['Load_SOCO_Temp_2035.dat'] = load_df['Load_SOCO_2035.dat'] * SOCO_LF
    load_df['Load_GULF_2035.dat'] = load_df['Load_SOCO_2035.dat'] * GULF_LF
    
    # Compute the loads for the SWPP subregions:
    load_df['Load_AEPW_2035.dat'] = load_df['Load_SWPP_2035.dat'] * AEPW_LF
    load_df['Load_GRDA_2035.dat'] = load_df['Load_SWPP_2035.dat'] * GRDA_LF
    load_df['Load_OKGE_2035.dat'] = load_df['Load_SWPP_2035.dat'] * OKGE_LF
    load_df['Load_WFEC_2035.dat'] = load_df['Load_SWPP_2035.dat'] * WFEC_LF
    load_df['Load_SPS_2035.dat'] = load_df['Load_SWPP_2035.dat'] * SPS_LF
    load_df['Load_OMPA_2035.dat'] = load_df['Load_SWPP_2035.dat'] * OMPA_LF
    load_df['Load_MIDW_2035.dat'] = load_df['Load_SWPP_2035.dat'] * MIDW_LF
    load_df['Load_SUNC_2035.dat'] = load_df['Load_SWPP_2035.dat'] * SUNC_LF
    load_df['Load_WERE_2035.dat'] = load_df['Load_SWPP_2035.dat'] * WERE_LF
    load_df['Load_KCPL_2035.dat'] = load_df['Load_SWPP_2035.dat'] * KCPL_LF
    load_df['Load_KACY_2035.dat'] = load_df['Load_SWPP_2035.dat'] * KACY_LF
    load_df['Load_EMDE_2035.dat'] = load_df['Load_SWPP_2035.dat'] * EMDE_LF
    load_df['Load_INDN_2035.dat'] = load_df['Load_SWPP_2035.dat'] * INDN_LF
    load_df['Load_SPRM_2035.dat'] = load_df['Load_SWPP_2035.dat'] * SPRM_LF
    load_df['Load_NPPD_2035.dat'] = load_df['Load_SWPP_2035.dat'] * NPPD_LF
    load_df['Load_GRIS_2035.dat'] = load_df['Load_SWPP_2035.dat'] * GRIS_LF
    load_df['Load_OPPD_2035.dat'] = load_df['Load_SWPP_2035.dat'] * OPPD_LF
    load_df['Load_WAPA_2035.dat'] = load_df['Load_SWPP_2035.dat'] * WAPA_LF
    load_df['Load_BEPC-SPP_2035.dat'] = load_df['Load_SWPP_2035.dat'] * BEPC_SPP_LF
    
    # Compute the loads for the TVA subregions:
    load_df['Load_TVA_Temp_2035.dat'] = load_df['Load_TVA_2035.dat'] * TVA_LF
    load_df['Load_SMT_2035.dat'] = load_df['Load_TVA_2035.dat'] * SMT_LF
    
    # Drop the un-needed columns for BAs with subregions:
    del load_df['Load_NEVP_2035.dat'], load_df['Load_FMPP_2035.dat'], load_df['Load_FPL_2035.dat'], load_df['Load_PJM_2035.dat'], load_df['Load_TVA_2035.dat'], load_df['Load_SWPP_2035.dat']
    del load_df['Load_CISO_2035.dat'], load_df['Load_IPCO_2035.dat'], load_df['Load_PACE_2035.dat'], load_df['Load_SOCO_2035.dat'], load_df['Load_MISO_2035.dat']
    
    # Clean up the redundant BA naming:
    load_df.rename(columns={'Load_NEVP_Temp_2035.dat': 'Load_NEVP_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_FMPP_Temp_2035.dat': 'Load_FMPP_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_FPL_Temp_2035.dat': 'Load_FPL_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_PJM_Temp_2035.dat': 'Load_PJM_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_SOCO_Temp_2035.dat': 'Load_SOCO_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_TVA_Temp_2035.dat': 'Load_TVA_2035.dat'}, inplace=True)
    
    # Read in the raw data GridView .csv file:
    raw_gv_df = pd.read_csv((data_dir + 'wecc_and_eic_load_2035.csv'), low_memory=False)
    
    # Delete the index column:
    del raw_gv_df["Index"] 
    
    # Subset to just the rows we need:
    raw_gv_df = raw_gv_df[1:8761]
    
    # Convert the values to floats:
    raw_gv_df = raw_gv_df.astype('float64')
    
    # Shift the index by -1:
    raw_gv_df = raw_gv_df.reset_index()
    
    # Merge in the GridView columns that aren't modeled by TELL:
    load_df = pd.concat([load_df,raw_gv_df['Load_SETH_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_SERU_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_SEHA_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_NF_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_IPP-REL_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_MH_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TH_Mead_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TH_Malin_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_SPC_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_CFE_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_CORNWALL_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_NS_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_IESO_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_BCHA_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_AESO_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_NB_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TE_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TH_PV_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_OSC_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_PS_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_MPW_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_GLH_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_CPLW_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_YAD_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_WECC_2035.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_WBDC-WECC_2035.dat']], axis=1)
    
    # Match the BA names in GridView:
    load_df.rename(columns={'Load_ERCO_2035.dat': 'Load_ERCOT_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_FPC_2035.dat': 'Load_DEF_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_TEC_2035.dat': 'Load_TECO_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_NYIS_2035.dat': 'Load_NYISO_2035.dat'}, inplace=True)
    load_df.rename(columns={'Load_ISNE_2035.dat': 'Load_ISO-NE_2035.dat'}, inplace=True)
        
    # Create a target dataframe with the spare hours:
    target_df = pd.DataFrame({"Index": np.arange(1,8791,1)})
    
    # Merge load dataframe with the target dataframe:
    merged_df = target_df.merge(load_df, on=['Index'], how='left')
    
    # Compute the summary statistics:
    stats_df = merged_df.apply(['mean','sum','max','min']).round(2)
    
    # Fix the summary statistic labels:
    stats_df.iloc[0, 0] = 'AVG'
    stats_df.iloc[1, 0] = 'SUM'
    stats_df.iloc[2, 0] = 'MAX'
    stats_df.iloc[3, 0] = 'MIN'
      
    # Sort the data by column name and make the Index column appear first:
    merged_df.rename(columns={'Index': 'AA'}, inplace=True)
    merged_df = merged_df.sort_index(axis = 1)
    merged_df.rename(columns={'AA': 'Index'}, inplace=True)
    
    # Add in a blank row and fill it with the year placeholder:
    merged_df.loc[-0.5] = 0
    merged_df = merged_df.sort_index().reset_index(drop=True)
    merged_df.iloc[0, :] = '2035'
    merged_df.at[0, 'Index'] = 'Year'
        
    # Merge the hourly load data and statistics dataframes together:
    output_df = pd.concat([merged_df, stats_df], axis=0)
    
    # Replace NaNs with blank values:
    output_df.replace(np.nan, "", regex=True)
        
    # Set the output filename:
    if year_to_process == '2015':
       output_filename = 'TELL_EIC_WECC_Loads_2035_Based_on_2015_Weather.csv'
    if year_to_process == '2018':
       output_filename = 'TELL_EIC_WECC_Loads_2035_Based_on_2018_Weather.csv'
    if year_to_process == '2055':
       output_filename = 'TELL_EIC_WECC_Loads_2035_Based_on_2015_Weather_With_Climate_Change.csv'
    if year_to_process == '2058':
       output_filename = 'TELL_EIC_WECC_Loads_2035_Based_on_2018_Weather_With_Climate_Change.csv'
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_dir, output_filename)), sep=',', index=False)
    
    # Return the output dataframe:
    return output_df


## Call the Necessary Functions to Process the Data:


In [ ]:
output_df = format_scaled_tell_loads(data_dir = data_dir,
                                     year_to_process = '2058')

output_df
